<a href="https://colab.research.google.com/github/univer-coder/Deep-learning-course/blob/main/lecture04_homework3_ac%3D08515.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第4回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール


- 訓練データは`x_train`， `t_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train`，`t_train`以外の学習データは使わないでください．**
- Pytorchを利用して構いません．
- ただし，**`torch.nn.Conv2d`のような高レベルのAPIは使用しないで下さい**．具体的には，`nn.Parameter`, `nn.Module`, `nn.Sequential`以外の`nn`系のAPIです．
- torchvision等で既に実装されているモデルも使用しないで下さい．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化」を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコードをコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`t_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### データの読み込み（この部分は修正しないでください）

`__len__`は，Pythonの組み込み関数len()を呼んだときに，内部で呼ばれる特殊メソッドです．

`__getitem__`は，インデックスやキーで要素を取得するときに，内部で呼ばれる特殊メソッドです．

どちらも， Datasetクラスを自作する際によく登場します．

```python
class MyList:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

mylist = MyList([10, 20, 30])
print(len(mylist))  # __len__が呼び出される
# 3
print(mylist[1])  # __getitem__が呼び出される
# 20
```

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect
import torch._dynamo
torch._dynamo.disable()

nn_except = ["Module", "Parameter", "Sequential"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0])

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#学習データ
x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture04/data/x_train.npy')
t_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture04/data/y_train.npy')

#テストデータ
x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture04/data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 多層パーセプトロンの実装

In [3]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [8]:
def relu(x):
    return torch.relu(x)


def cross_entropy_loss(y_logit, t_hot):
    logsumexp = torch.logsumexp(y_logit, dim=1, keepdim=True)
    log_probs = y_logit - logsumexp
    return -(t_hot * log_probs).sum(dim=1).mean()


# 過学習を防ぐdropout
class Dropout(nn.Module):
    def __init__(self, p=0.1):
        super().__init__()
        self.p = p

    def forward(self, x):
        if self.training:
            mask = (torch.rand_like(x) > self.p).float()
            return x * mask / (1.0 - self.p)
        else:
            return x


class Dense(nn.Module):  # nn.Moduleを継承する
    def __init__(self, in_dim, out_dim, function=lambda x: x):
        super().__init__()
        self.W = nn.Parameter(torch.tensor(np.random.uniform(
                        low=-np.sqrt(6/in_dim),
                        high=np.sqrt(6/in_dim),
                        size=(in_dim, out_dim)
                    ), dtype=torch.float32))
        self.b = nn.Parameter(torch.tensor(np.zeros([out_dim]), dtype=torch.float32))
        self.function = function

    def forward(self, x):
        return self.function(torch.matmul(x, self.W) + self.b)


class MLP(nn.Module):
    def __init__(self, in_dim, hid_dims, out_dim, dropout_p=0.1):
        super(MLP, self).__init__()
        layers = []
        input_dim = in_dim

        for hid_dim in hid_dims:
            layers.append(Dense(input_dim, hid_dim, function=relu))
            layers.append(Dropout(p=dropout_p))
            input_dim = hid_dim

        layers.append(Dense(input_dim, out_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

n_epochs = 30
drop_rate = 0.1

in_dim = 784
out_dim = 10
hid_dims = [256, 128, 64]
lr = 0.0005
mlp = MLP(in_dim=784, hid_dims=hid_dims, out_dim=10, dropout_p=0.1).to(device)
optimizer = torch.optim.Adam(mlp.parameters(), lr=lr, weight_decay=0.01)

best_val_accuracy = 0
patience = 10
patience_counter = 0
best_weights = None
one_hot = torch.eye(10, device=device)

In [9]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        t_hot = one_hot[t]

        # テンソルをGPUに移動
        x = x.to(device)
        t_hot = t_hot.to(device)

        # 順伝播　勾配計算なし
        y = mlp(x)

        # 誤差の計算(クロスエントロピー誤差関数)
        loss = cross_entropy_loss(y, t_hot)

        # 誤差の逆伝播
        optimizer.zero_grad()
        loss.backward()

        # パラメータの更新
        optimizer.step()

        # モデルの出力を予測値のスカラーに変換
        pred = y.argmax(1)

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    mlp.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        t_hot = one_hot[t]  # 正解ラベルをone-hot vector化

        # テンソルをGPUに移動
        x = x.to(device)
        t_hot = t_hot.to(device)

        # 順伝播　勾配計算なし
        with torch.no_grad():
            y = mlp(x)

            # 誤差の計算(クロスエントロピー誤差関数)
            loss = cross_entropy_loss(y, t_hot)

            # モデルの出力を予測値のスカラーに変換
            pred = y.argmax(1)

            losses_valid.append(loss.tolist())

            acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
            valid_num += acc.size()[0]
            valid_true_num += acc.sum().item()

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        train_true_num/train_num,
        np.mean(losses_valid),
        valid_true_num/valid_num
    ))

    # Early stopping 条件のチェック
    current_val_accuracy = valid_true_num / valid_num
    if current_val_accuracy > best_val_accuracy:
        best_val_accuracy = current_val_accuracy
        best_weights = [p.detach().clone() for p in mlp.parameters()]
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break


# 一番良かった重みに戻す
if best_weights:
    with torch.no_grad():
        for p, best_p in zip(mlp.parameters(), best_weights):
            p.copy_(best_p)

EPOCH: 0, Train [Loss: 0.652, Accuracy: 0.773], Valid [Loss: 0.540, Accuracy: 0.821]
EPOCH: 1, Train [Loss: 0.547, Accuracy: 0.811], Valid [Loss: 0.476, Accuracy: 0.836]
EPOCH: 2, Train [Loss: 0.533, Accuracy: 0.816], Valid [Loss: 0.510, Accuracy: 0.821]
EPOCH: 3, Train [Loss: 0.526, Accuracy: 0.817], Valid [Loss: 0.484, Accuracy: 0.836]
EPOCH: 4, Train [Loss: 0.522, Accuracy: 0.820], Valid [Loss: 0.481, Accuracy: 0.832]
EPOCH: 5, Train [Loss: 0.516, Accuracy: 0.823], Valid [Loss: 0.515, Accuracy: 0.812]
EPOCH: 6, Train [Loss: 0.512, Accuracy: 0.824], Valid [Loss: 0.470, Accuracy: 0.836]
EPOCH: 7, Train [Loss: 0.512, Accuracy: 0.825], Valid [Loss: 0.480, Accuracy: 0.831]
EPOCH: 8, Train [Loss: 0.503, Accuracy: 0.828], Valid [Loss: 0.466, Accuracy: 0.835]
EPOCH: 9, Train [Loss: 0.501, Accuracy: 0.828], Valid [Loss: 0.480, Accuracy: 0.828]
EPOCH: 10, Train [Loss: 0.504, Accuracy: 0.825], Valid [Loss: 0.458, Accuracy: 0.840]
EPOCH: 11, Train [Loss: 0.497, Accuracy: 0.829], Valid [Loss: 0.

In [10]:
mlp.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = mlp.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture04/submission3_pred.csv', header=True, index_label='id')